<a href="https://colab.research.google.com/github/lakshaychitkara/2210990536/blob/main/Research-paper-RAGed-Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##setup

In [ ]:
!pip -q install -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu datasets evaluate nltk pandas tqdm

import os, re, time, math, json
import pandas as pd
from tqdm import tqdm

import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 13.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 3.0.1 which is incompatible.
cudf-cu12 25.10.0 requ

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Load dataset (SciFact-style: question + answer + optional evidence)
# This uses a lightweight public dataset via datasets. If you want your own dataset later, you’ll just supply the same schema.

In [ ]:
from datasets import load_dataset

# Small, fast demo dataset
# We'll use "scifact" (question/claim + abstracts) style using the BEIR variant isn't directly in HF in one format,
# so we will use a simpler QA dataset for correctness metrics and treat retrieved docs as citations.
# Recommended for "evidence doc id" experiments: later swap to BEIR SciFact with explicit corpus/qrels.

ds = load_dataset("nq_open", split="train[:200]")  # keep small for Colab
ds = ds.map(lambda x: {"question": x["question"], "answer": x["answer"][0] if len(x["answer"]) else ""})
ds[0]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

nq_open/train-00000-of-00001.parquet:   0%|          | 0.00/4.46M [00:00<?, ?B/s]

nq_open/validation-00000-of-00001.parque(…):   0%|          | 0.00/214k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87925 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3610 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

{'question': 'where did they film hot tub time machine',
 'answer': 'Fernie Alpine Resort'}

Cell 3 — Build a tiny document corpus

For controlled experiments, you need a fixed corpus. For demo, we’ll build a corpus from Wikipedia-like snippets shipped in HF datasets (fast).
Later, you can replace this with your paper PDFs, website dumps, or your own dataset corpus.

In [ ]:
from datasets import load_dataset

# Modern Wikipedia dataset (no script error)
corpus_ds = load_dataset(
    "wikimedia/wikipedia",
    "20231101.en",
    split="train[:2000]"
)

raw_docs = [
    d["text"].replace("\n", " ").strip()
    for d in corpus_ds
    if d.get("text")
]

raw_docs = [t for t in raw_docs if len(t) > 400]
raw_docs = raw_docs[:1500]

print(len(raw_docs))
print(raw_docs[0][:300])

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

20231101.en/train-00000-of-00041.parquet:   0%|          | 0.00/420M [00:00<?, ?B/s]

20231101.en/train-00001-of-00041.parquet:   0%|          | 0.00/351M [00:00<?, ?B/s]

20231101.en/train-00002-of-00041.parquet:   0%|          | 0.00/329M [00:00<?, ?B/s]

...

20231101.en/train-00038-of-00041.parquet:   0%|          | 0.00/538M [00:00<?, ?B/s]

20231101.en/train-00039-of-00041.parquet:   0%|          | 0.00/465M [00:00<?, ?B/s]

20231101.en/train-00040-of-00041.parquet:   0%|          | 0.00/422M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6407814 [00:00<?, ? examples/s]

1500
Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typically including nation-states, and capitalism. Anarchism advocates for the replacement of the state 


Cell 4 — Chunking functions

In [ ]:
def chunk_fixed(text, chunk_size=512, overlap=64):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = " ".join(words[i:i+chunk_size])
        if chunk.strip():
            chunks.append(chunk)
        i += max(1, chunk_size - overlap)
    return chunks

def chunk_sentence(text, max_words=220):
    sents = sent_tokenize(text)
    chunks, buf = [], []
    buf_words = 0
    for s in sents:
        w = s.split()
        if buf_words + len(w) > max_words and buf:
            chunks.append(" ".join(buf))
            buf, buf_words = [], 0
        buf.append(s)
        buf_words += len(w)
    if buf:
        chunks.append(" ".join(buf))
    return chunks

def build_chunks(corpus_texts, strategy):
    all_chunks = []
    for doc_id, text in enumerate(corpus_texts):
        if strategy["type"] == "fixed":
            chunks = chunk_fixed(text, strategy["chunk_size"], strategy["overlap"])
        elif strategy["type"] == "sentence":
            chunks = chunk_sentence(text, strategy["max_words"])
        else:
            raise ValueError("Unknown chunking strategy")
        for ci, c in enumerate(chunks):
            all_chunks.append({
                "doc_id": doc_id,
                "chunk_id": f"{doc_id}_{ci}",
                "text": c
            })
    return all_chunks

Cell 5 — Embeddings + FAISS index builder

In [ ]:
embed_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(embed_model_name)

def build_faiss_index(chunks):
    texts = [c["text"] for c in chunks]
    embs = embedder.encode(texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
    dim = embs.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embs)
    return index, embs

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Cell 5 — Embeddings + FAISS index builder

In [ ]:
embed_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(embed_model_name)

def build_faiss_index(chunks):
    texts = [c["text"] for c in chunks]
    embs = embedder.encode(texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
    dim = embs.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embs)
    return index, embs

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Cell 6 — Retriever + optional reranker

In [ ]:
reranker_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
reranker = CrossEncoder(reranker_name)

def retrieve(query, index, chunks, top_k=5):
    q_emb = embedder.encode([query], normalize_embeddings=True, convert_to_numpy=True)
    scores, ids = index.search(q_emb, top_k)
    hits = []
    for score, idx in zip(scores[0], ids[0]):
        c = chunks[int(idx)]
        hits.append({**c, "score": float(score)})
    return hits

def rerank(query, hits, top_k=5):
    pairs = [(query, h["text"]) for h in hits]
    rr_scores = reranker.predict(pairs)
    reranked = []
    for h, s in zip(hits, rr_scores):
        reranked.append({**h, "rr_score": float(s)})
    reranked.sort(key=lambda x: x["rr_score"], reverse=True)
    return reranked[:top_k]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Cell 7 — Load a small instruct LLM (4-bit)

This runs on typical Colab T4/L4. If you get OOM, reduce model size.

In [ ]:
model_name = "Qwen/Qwen2.5-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.eval()

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm

Cell 8 — Prompts + generation wrapper (with citation format)

We’ll force citations like [1], [2] pointing to retrieved chunks.

In [ ]:
def build_prompt(question, contexts, tactic="normal"):
    # contexts: list of dicts with "text"
    # Create numbered sources
    sources = []
    for i, c in enumerate(contexts, start=1):
        sources.append(f"[{i}] {c['text']}")
    sources_block = "\n\n".join(sources)

    if tactic == "normal":
        instr = (
            "Answer the question using the sources below. "
            "If unsure, say you don't know. Add citations like [1] after the sentences they support."
        )
    elif tactic == "strict":
        instr = (
            "You MUST answer using ONLY the sources below. "
            "If the answer is not explicitly stated, reply exactly: \"I don't know based on the provided sources.\" "
            "Every factual sentence must end with at least one citation like [1]."
        )
    else:
        raise ValueError("Unknown tactic")

    user = f"Question: {question}\n\nSources:\n{sources_block}\n\n{instr}\nAnswer:"
    return user

@torch.no_grad()
def generate_answer(prompt, decoding):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    t0 = time.perf_counter()

    out = model.generate(
        **inputs,
        max_new_tokens=decoding["max_new_tokens"],
        do_sample=decoding["temperature"] > 0,
        temperature=decoding["temperature"],
        top_p=decoding["top_p"],
        repetition_penalty=1.05,
        pad_token_id=tokenizer.eos_token_id
    )

    latency = time.perf_counter() - t0
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # Keep only after "Answer:" (best-effort)
    ans = text.split("Answer:")[-1].strip()
    prompt_tokens = int(inputs["input_ids"].shape[1])
    gen_tokens = int(out.shape[1] - inputs["input_ids"].shape[1])
    return ans, latency, prompt_tokens, gen_tokens

def extract_citations(answer_text):
    # returns cited source indices like [1], [2]
    cites = re.findall(r"\[(\d+)\]", answer_text)
    return [int(c) for c in cites if c.isdigit()]

Cell 9 — Metrics (quality + citation accuracy + basic faithfulness proxy)

For demo:

Quality: token-level F1 (SQuAD-like)

Citation accuracy: proportion of cited ids that are valid, plus “has any citation”

Faithfulness proxy (cheap): answer sentences that have at least one citation

In [ ]:
squad_metric = evaluate.load("squad")

def squad_like_f1(pred, gold):
    # evaluate.squad expects list of dicts
    refs = [{"id":"0", "answers":{"text":[gold], "answer_start":[0]}}]
    preds = [{"id":"0", "prediction_text": pred}]
    return squad_metric.compute(predictions=preds, references=refs)["f1"]

def faithfulness_proxy(answer):
    sents = [s.strip() for s in re.split(r"[.\n]", answer) if s.strip()]
    if not sents:
        return 0.0
    cited = 0
    for s in sents:
        if re.search(r"\[\d+\]", s):
            cited += 1
    return cited / len(sents)

def citation_metrics(answer, n_sources):
    cites = extract_citations(answer)
    if len(cites) == 0:
        return {"has_citation": 0, "valid_cite_rate": 0.0}
    valid = sum(1 for c in cites if 1 <= c <= n_sources)
    return {
        "has_citation": 1,
        "valid_cite_rate": valid / len(cites)
    }

Cell 10 — Experiment grid + runner

In [ ]:
chunking_grid = [
    {"name":"fixed_512", "type":"fixed", "chunk_size":512, "overlap":64},
    {"name":"fixed_256", "type":"fixed", "chunk_size":256, "overlap":32},
    {"name":"sentence_220w", "type":"sentence", "max_words":220},
]

retrieval_grid = [
    {"top_k": 3},
    {"top_k": 5},
    {"top_k": 10},
]

rerank_grid = [
    {"rerank": False},
    {"rerank": True},
]

prompt_grid = [
    {"tactic": "normal"},
    {"tactic": "strict"},
]

decoding_grid = [
    {"name":"deterministic", "temperature":0.0, "top_p":1.0, "max_new_tokens":160},
    {"name":"balanced", "temperature":0.4, "top_p":0.9, "max_new_tokens":160},
]

def run_experiment(ds, corpus_texts, n_questions=50):
    rows = []
    sample = ds.select(range(min(n_questions, len(ds))))

    for ch in chunking_grid:
        print(f"\n--- Building chunks & index: {ch['name']} ---")
        chunks = build_chunks(corpus_texts, ch)
        index, _ = build_faiss_index(chunks)

        for r in retrieval_grid:
            for rr in rerank_grid:
                for pg in prompt_grid:
                    for dec in decoding_grid:
                        config_id = f"{ch['name']}|k{r['top_k']}|rr{int(rr['rerank'])}|{pg['tactic']}|{dec['name']}"
                        print(f"\nRunning: {config_id}")

                        for ex in tqdm(sample, desc=config_id):
                            q = ex["question"]
                            gold = ex["answer"]

                            t_retr0 = time.perf_counter()
                            hits = retrieve(q, index, chunks, top_k=r["top_k"])
                            retr_latency = time.perf_counter() - t_retr0

                            if rr["rerank"]:
                                t_rr0 = time.perf_counter()
                                hits = rerank(q, hits, top_k=r["top_k"])
                                rr_latency = time.perf_counter() - t_rr0
                            else:
                                rr_latency = 0.0

                            prompt = build_prompt(q, hits, tactic=pg["tactic"])
                            pred, gen_latency, p_tok, g_tok = generate_answer(prompt, dec)

                            f1 = squad_like_f1(pred, gold)
                            faith = faithfulness_proxy(pred)
                            cmet = citation_metrics(pred, n_sources=len(hits))

                            rows.append({
                                "config": config_id,
                                "question": q,
                                "gold": gold,
                                "pred": pred,
                                "f1": f1,
                                "faithfulness_proxy": faith,
                                "has_citation": cmet["has_citation"],
                                "valid_cite_rate": cmet["valid_cite_rate"],
                                "retrieval_latency_s": retr_latency,
                                "rerank_latency_s": rr_latency,
                                "gen_latency_s": gen_latency,
                                "prompt_tokens": p_tok,
                                "gen_tokens": g_tok,
                                "total_latency_s": retr_latency + rr_latency + gen_latency
                            })

    return pd.DataFrame(rows)

df = run_experiment(ds, raw_docs, n_questions=30)
df.head()

Cell 11 — Aggregate results + save

In [ ]:
agg = (df.groupby("config")
       .agg(
           mean_f1=("f1","mean"),
           mean_faith=("faithfulness_proxy","mean"),
           citation_presence=("has_citation","mean"),
           mean_valid_cite=("valid_cite_rate","mean"),
           mean_total_latency=("total_latency_s","mean"),
           mean_prompt_tokens=("prompt_tokens","mean"),
           mean_gen_tokens=("gen_tokens","mean"),
       )
       .reset_index()
       .sort_values(["mean_f1","mean_faith","mean_valid_cite","mean_total_latency"], ascending=[False,False,False,True]))

display(agg.head(20))

df.to_csv("rag_design_study_runs.csv", index=False)
agg.to_csv("rag_design_study_summary.csv", index=False)

print("Saved: rag_design_study_runs.csv, rag_design_study_summary.csv")

2) What you do after running this code

Start with the demo (as-is) and confirm it runs end-to-end.

You’ll get rag_design_study_runs.csv (per-question) and rag_design_study_summary.csv (per-config).

Switch from the demo corpus to your real corpus (papers/articles).

Replace raw_docs with your documents:

If you have PDFs: extract text (PyMuPDF) → list of strings.

If you have web pages: store cleaned text per page.

Switch from the demo QA dataset to your evaluation datasets.

Make a CSV with columns: question, answer (and optionally gold_doc_id).

Load it in Colab and convert to the same schema used in the notebook.

Add interaction analyses (your “pairwise interaction effects” objective).

Keep the grid small (e.g., chunking × top_k) and plot heatmaps:

mean_f1 vs (chunking, top_k)

mean_latency vs (rerank, top_k)

Upgrade “faithfulness/factuality” measurement for your paper.

Keep the cheap proxy in the tables (it’s useful), but add:

an LLM-as-a-judge faithfulness score (open-source judge or API),

and a citation-to-evidence alignment check (sentence-level entailment, or judge).

For a proper RAG paper, you should eventually move to:

BEIR SciFact

BEIR Natural Questions

HotpotQA

your own domain corpus